## Equations of Motion Given

In [1]:
# Import everything we need from SymPy
import sympy as sym
import numpy as np
from sympy import Symbol, Matrix
from sympy import N as N_sympy
from sympy import simplify, sin, cos, init_printing
init_printing() # This function will make the outputs of SymPy look prettier and be easier to read

# Constants of the system
m_c = 12.0    # Mass of the chassis in kg
ixx_c = 1.0   # Roll mass moment of ineria of the chassis
iyy_c = 0.8   # Pitch mass moment of interia of the chassis
izz_c = 0.52  # Yaw mass moment of interia of the chassis

m_w = 1.2       # Mass of either wheel in kg
iaa_w = 0.0634  # Axial mass moment of interia of the wheels
itt_w = 0.0327  # Transverse mass moment of interia of the wheels
r_w = 0.325     # Radius of either wheel in meters

m = m_c + 2*m_w        # The total mass of the system
ixx = ixx_c + 2*itt_w  # The total roll mass moment of inertia of the system
iyy = iyy_c            # The total pitch mass moment of inertia of the system
izz = izz_c + 2*itt_w  # The total yaw mass moment of inertia of the system

h = 0.2   # Distance from the CoM of the chassis to wheel axle in meters
a = 0.35  # Half the distance between the wheels in meters

r_station = 20.0  # The radius of the station in meters
v_station = -0.1  # The angular velocity of the station in rad/second

# Create the system's generalized coordinates 
e_lat = Symbol('e_lat')          # The lateral error of the segbot. Positive is to the left of the target position.
phi = Symbol('phi')              # The yaw of the chassis of the segbot relative to the station. Positive is facing left.
phidot = Symbol('phidot')        # The yawing velocity of the segbot relative to the station. Positive is turning left.
v = Symbol('v')                  # The forward velocity of the segbot. Positive is moving forward.
theta = Symbol('theta')          # The pitch of the chassis of the segbot. Positive is tipped forwards.
thetadot = Symbol('thetadot')    # The pitching velocity of the segbot. Positive is tipping forward.
tau_l = Symbol('tau_l')          # The torque applied to the left wheel of the segbot. Positive moves the segbot forward.
tau_r = Symbol('tau_r')          # The torque applied to the right wheel of the segbot. Positive moves the segbot forward.

M = Matrix([[m + 2 * iaa_w / r_w**2, 0, m_c * h * cos(theta)],
            [0, (ixx + m_c * h**2) * sin(theta)**2 + izz * cos(theta)**2 + (2 * iaa_w * a**2 / r_w**2) + 2 * m_w * a**2, 0],
            [m_c * h * cos(theta), 0, iyy * m_c * h**2]])
N = Matrix([[m_c * h * (phidot**2 + thetadot**2) * sin(theta)],
            [-2 * (ixx - izz + m * h**2) * cos(theta) * sin(theta) * phidot * thetadot - m_c * h * sin(theta) * v * phidot],
            [(ixx - izz + m_c * h**2) * cos(theta) * sin(theta) * phidot**2 + m_c * 9.81 * h * sin(theta)]])
R = Matrix([[1 / r_w, 1 / r_w],
            [-a / r_w, a / r_w],
            [-1, -1]])
f_partial = simplify(M.inv() * (N + R * Matrix([[tau_l], [tau_r]])))
f = Matrix([[v * sin(phi)],
            [f_partial]])
f

⎡                                                                             
⎢                                                                             
⎢                                                                             
⎢                -18.4325593081475⋅τₗ - 18.4325593081475⋅τᵣ - 14.377396260355⋅
⎢                ─────────────────────────────────────────────────────────────
⎢                                                                             
⎢                                                                     89.85872
⎢                                                                             
⎢                                                  -1.056⋅φ̇⋅θ̇⋅sin(2⋅θ) - 2.4
⎢                                                  ───────────────────────────
⎢                                                                             
⎢                                                                          0.9
⎢                                                   

## System Linearization and State Space Form

In [110]:
# FIXME: Find an equilibrium point and linearize the system into state space form
# Generalized coordinates
e_lat, phi, phidot, v, theta, thetadot, tau_l, tau_r = sym.symbols('e_lat phi phidot v theta thetadot tau_l tau_r')

# Compute the Jacobians
state_vars = sym.Matrix([e_lat, v, phi, phidot, theta, thetadot])
inputs = sym.Matrix([tau_l, tau_r])

# Jacobian matrices
A = f.jacobian(state_vars)
B = f.jacobian(inputs)

# Define equilibrium point: theta = 0, phi = 0, all velocities = 0, torques = 0
equilibrium_point = {
    theta: 0,
    thetadot: 0,
    phi: 0,
    phidot: 0,
    v: 0,
    tau_l: 0,
    tau_r: 0
}

# Substitute equilibrium values into the Jacobians A and B
A_eq = A.subs(equilibrium_point)
B_eq = B.subs(equilibrium_point)

A_eq, B_eq



⎛⎡0  0  0  0          0          0  0  0⎤                                     
⎜⎢                                      ⎥                                     
⎜⎢0  0  0  0  -245.056661411115  0  0  0⎥                                     
⎜⎢                                      ⎥  ⎡        0                  0      
⎜⎢0  0  0  0          0          0  0  0⎥  ⎢                                  
⎜⎢                                      ⎥  ⎢15.5326172644856   15.532617264485
⎜⎢0  0  0  0  1592.91663381947   0  0  0⎥  ⎢                                  
⎜⎢                                      ⎥, ⎢-1.04916430028235  1.0491643002823
⎜⎢0  0  0  0          0          0  0  0⎥  ⎢                                  
⎜⎢                                      ⎥  ⎢-99.6830245697014  -99.68302456970
⎜⎢0  0  0  0          0          0  0  0⎥  ⎢                                  
⎜⎢                                      ⎥  ⎣        0                  0      
⎜⎢0  0  0  0          0          0  0  0⎥           

## FIXME: Gain Selection and Controller Design

In [3]:
# FIXME: Select a set of control and observer gains. Ensure the resultant system is stable. Implement your controller below.

# PID controller parameters for pitch (theta) and yaw (phi)
Kp_theta = 50.0  # Proportional gain for pitch
Ki_theta = 1.0   # Integral gain for pitch
Kd_theta = 2.0   # Derivative gain for pitch

Kp_phi = 50.0    # Proportional gain for yaw
Ki_phi = 1.0     # Integral gain for yaw
Kd_phi = 2.0     # Derivative gain for yaw

In [101]:
class Controller():
    def __init__(self):
        # PID gains for pitch control (theta)
        self.Kp_theta = 1000.0
        self.Ki_theta = 5.0
        self.Kd_theta = 100.0
        
        # PID gains for yaw control (phi)
        self.Kp_phi = 50.0
        self.Ki_phi = 10.0
        self.Kd_phi = 20.0
        
        self.K_e = 10.0

        # Integral terms
        self.integral_theta = 0.0
        self.integral_phi = 0.0
        
        # Previous errors for derivative terms
        self.previous_error_theta = 0.0
        self.previous_error_phi = 0.0

    def reset(self):
        # Reset integral and previous error terms
        self.integral_theta = 0.0
        self.integral_phi = 0.0
        self.previous_error_theta = 0.0
        self.previous_error_phi = 0.0

    def run(self, **kwargs):
        # Target setpoints (desired values)
        target_theta = 0.0  # Desired pitch
        target_phi = 0.0    # Desired yaw

        # Extract current states from kwargs
        current_theta = kwargs["theta"]
        current_phidot = kwargs["phidot"]
        current_thetadot = kwargs["thetadot"]
        current_phi = kwargs["phi"]
        current_e_lat = kwargs["e_lat"]

        # Time step (assuming a fixed time step)
        dt = 0.01  # Example value (this should ideally be dynamically set)

        # Update desired pitch based on lateral error (e_lat)
        target_phi = self.K_e * current_e_lat  # Adjust desired pitch based on e_lat

        # Compute errors
        error_theta = target_theta - current_theta
        error_phi = target_phi - current_phi

        # Update integral terms
        self.integral_theta += error_theta * dt
        self.integral_phi += error_phi * dt

        # Compute derivative terms
        derivative_theta = (error_theta - self.previous_error_theta) / dt
        derivative_phi = (error_phi - self.previous_error_phi) / dt

        # Compute control signals (torques) using PID
        tau_l_theta = (self.Kp_theta * error_theta +
                       self.Ki_theta * self.integral_theta +
                       self.Kd_theta * derivative_theta)
        tau_r_theta = tau_l_theta  # Both wheels are affected equally by pitch

        tau_l_phi = (self.Kp_phi * error_phi +
                     self.Ki_phi * self.integral_phi +
                     self.Kd_phi * derivative_phi)
        tau_r_phi = -tau_l_phi  # Yaw control affects wheels oppositely

        # Combine torques for pitch and yaw
        left_torque = tau_l_theta + tau_l_phi
        right_torque = tau_r_theta - tau_r_phi

        # Update previous errors for the next step
        self.previous_error_theta = error_theta
        self.previous_error_phi = error_phi

        # Return the calculated torques for the left and right wheels
        inputs = [-left_torque, -right_torque]
        return inputs

In [102]:
# Create an instance of our Controller class
controller = Controller()

## Running the Simulation

Before we run the simulation, we must first import the simulation module and then create an instance of the simulator. This is done below:

In [103]:
# Import the segbot project. This module is used to simulate, render, and plot a segbot dynamic system
import ae353_segbot

In [104]:
sim = ae353_segbot.Segbot_sim(use_keyboard=True,
                              visualization=True,
                              animation=False,
                              bumpy=False)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7020/static/


In [105]:
# Run the simulation and collect the simulation data
data = sim.run(controller,
               max_time=None,
               initial_e_lat = 0.0,
               station_velocity = -0.1)

PRESS ENTER TO START SIMULATION.
PRESS ESC TO QUIT.
PRESS SPACE TO PAUSE/RESUME SIMULATION.
PRESS BACKSPACE TO RESET SIMULATION.
CONTINUING...
QUITTING...


Termination command detected. Terminating keyboard listener. Goodbye


## FIXME: Post-processing the simulation data

After the simulation is run, we can plot the data using the Matplotlib package. To start, we import matplotlib and numpy.

In [95]:
# Import the pyplot module from the matplotlib package
import matplotlib.pyplot as plt

# Import numpy
import numpy as np

Termination command detected. Terminating keyboard listener. Goodbye


Now we can make whichever plots we see fit to make sense of the simulation data. Here we plot the pitch versus time as well as the applied torques versus time.

**THIS IS LEFT INTENTIONALLY AS INSUFFICIENT TO PROVE YOUR CONTROLLER CAN STEER THE SEGBOT.**

In [ ]:
# Create a figure that has two plots in it
fig, axes = plt.subplots(2, 1, figsize=(7.2, 5.4), sharex=True)

# Create a plot of the pitch angle versus time on the first plot
axes[0].plot(data['time'], 180.*np.array(data['theta'])/np.pi, c='k', lw=2.5)
axes[0].set_ylabel("Pitch [deg]", fontsize=12)
axes[0].tick_params(axis='y', labelsize=12)
axes[0].set_title("Pitch vs. Time", fontsize=14)
axes[0].grid()

# Create a plot of the torque versus time on the second plot
axes[1].plot(data['time'], data['tau_l'], label="Left Torque", c='r', lw=2.5)
axes[1].plot(data['time'], data['tau_r'], label="Right Torque", c='b', lw=2.5)
axes[1].set_xlabel("Time [s]", fontsize=12)
axes[1].set_ylabel("Torque [Nm]", fontsize=12)
axes[1].tick_params(axis='y', labelsize=12)
axes[1].legend(fontsize=12, shadow=True)
axes[1].set_title("Torque vs. Time", fontsize=14)
axes[1].grid()

# Show the figure
fig.tight_layout(pad=2.0)
plt.show()